In [1]:
from pydub import AudioSegment

In [14]:
%%capture
%pip install pocketsphinx

In [31]:
MAPPING = """
ɑː AA
æ AE
ə AH
ɐ AH
ʌ AH
ɔː AO
aʊ AW
aɪ AY
b B
tʃ CH
d D
ð DH
ɛ EH
ɚ ER
ɜː ER
eɪ EY
f F
ɡ G
h HH
ɪ IH
i IY
iː IY
dʒ JH
k K
l L
m M
n N
ŋ NG
oʊ OW
ɔɪ OY
p P
ɹ R
s S
ʃ SH
t T
θ TH
ʊ UH
uː UW
v V
w W
j Y
z Z
ʒ ZH
ɾ D
"""

In [62]:
espeak_to_cmudict = {}
for line in MAPPING.split("\n"):
    if line == "":
        continue
    line = line.strip()
    parts = line.split(" ")

    if len(parts) != 2:
        print(line)
        continue
    k, v = line.split(" ")
    if not k in espeak_to_cmudict:
        espeak_to_cmudict[k] = v
    

In [81]:
import re

cmudict_keys = espeak_to_cmudict.keys()
cmudict_keys = sorted(cmudict_keys, key=len, reverse=True)
espeak_regex = re.compile(rf"({'|'.join(cmudict_keys)})")

def cmudictify(espeak):
    espeak = espeak.replace("ˈ", "").replace("ˌ", "")
    return " ".join([espeak_to_cmudict[x] for x in re.findall(espeak_regex, espeak)])

In [58]:
EGTEXT = "Yeah, that's true. I mean, they are the same size and they are a little bit, but I think I I should go more for something that style."
EGPHON = "/jˈæ ðˈæs tɹˈuː ə mˈiːn ðˈeɪ ɚ ðə sˈeɪm sˈaɪz ən ðˈeɪ ˈɑːɹ ə lˈɪɾə bˈɪɾ bˈʌt ˈaɪ θˈɪŋk ˈaɪ ˈaɪ ʃˈʊ ɡˈoʊ mˈɔːɹ fɚ sˈʌmθɪŋ ðˈæt stˈaɪl./"
EGFILE = "/Users/joregan/Playing/hsi/audio/hsi_7_0719_210_001_main.wav"
EGSTART = 70.028
EGEND = 75.441

In [79]:
def normword(text):
    text = text.strip(",.;:!?")
    return text.lower()

def normphon(phon):
    phon = phon.strip(",.;:!?")
    return phon

def make_lexicon(text, phon):
    if phon.startswith("/") and phon.endswith("/"):
        phon = phon[1:-1]
    words = [normword(x) for x in text.split(" ")]
    phonwords = [cmudictify(normphon(x)) for x in phon.split(" ")]
    assert len(words) == len(phonwords)
    output = list(set(zip(words, phonwords)))
    return output
    

In [83]:
lex = make_lexicon(EGTEXT, EGPHON)

In [85]:
audio = AudioSegment.from_file(EGFILE)

In [86]:
audio = audio.set_frame_rate(16000)

In [87]:
seg = audio[int(EGSTART * 1000):int(EGEND * 1000)]

In [90]:
seg.get_array_of_samples()

In [128]:
def make_ps_dict(entries):
    counts = {}
    output = []
    lex = sorted(entries)
    for entry in lex:
        count = 1
        if not entry[0] in counts:
            counts[entry[0]] = 1
        else:
            counts[entry[0]] += 1
            count = counts[entry[0]]
        if count != 1:
            subscript = f"({count})"
        else:
            subscript = ""
        output.append(f"{entry[0]}{subscript} {entry[1]}")
    return output

In [97]:
def make_fsg_transitions_from_text(text):
    words = [normword(x) for x in text.split(" ")]
    enum = [x for x in enumerate(words)]
    trans = [(x[0], x[0] + 1, 1.0, x[1]) for x in enum]
    return trans

In [99]:
fsgt = make_fsg_transitions_from_text(EGTEXT)

In [100]:
start_state = fsgt[0][0]
end_state = fsgt[-1][1]

In [125]:
import pocketsphinx

decoder = pocketsphinx.Decoder(lm=None)
fsg = decoder.create_fsg("dummy", start_state, end_state, fsgt)

In [132]:
import tempfile
entries = make_ps_dict(lex)

with tempfile.NamedTemporaryFile(suffix=".dict") as dictf:
    with open(dictf.name, "w") as dictout:
        for entry in entries:
            dictout.write(entry + "\n")

    decoder = pocketsphinx.Decoder(lm=None, dict=dictf.name)
    fsg = decoder.create_fsg("dummy", start_state, end_state, fsgt)

In [133]:
import tempfile
entries = make_ps_dict(lex)

with open("/tmp/testing.dict", "w") as dictout:
    for entry in entries:
        dictout.write(entry + "\n")

decoder = pocketsphinx.Decoder(lm=None, dict="/tmp/testing.dict")
fsg = decoder.create_fsg("dummy", start_state, end_state, fsgt)

In [134]:
decoder.process_raw(seg.get_array_of_samples())

ValueError: Buffer dtype mismatch, expected 'const unsigned char' but got 'int'